<a href="https://colab.research.google.com/github/MLo7Ghinsan/DiffSinger_colab_notebook_MLo7/blob/main/DiffSinger_colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# _**[DiffSinger](https://github.com/openvpi/DiffSinger)**_
_Singing Voice Synthesis via Shallow Diffusion Mechanism (SVS & TTS)_

\
____

Note:
- This notebook will get update semi-frequently based from the feedback or response from users

```If it’s stupid but it works, it’s not stupid```

\
____
\
#### **This notebook is an edited copy of Kei's Diffsinger [colab notebook](https://colab.research.google.com/drive/1kUg9dz8PPH92NfnLZwgq0_9B9an39t1J?usp=sharing)**
####**This notebook is maintained by MLo7**
\
___

# **Setup**

In [ ]:
#@title # Mount Google Drive and Setup


from IPython.display import clear_output
from IPython.display import Audio, display, HTML
import os
from google.colab import drive
drive.mount("/content/drive")

if not os.path.exists("/content/pretrain_models"):
    os.makedirs("/content/pretrain_models")

%cd /content
!wget https://github.com/MLo7Ghinsan/DiffSinger_colab_notebook_MLo7/releases/download/OU_files/jpn_dict.txt

!rm -rf /content/sample_data
!apt-get install aria2
clear_output()

!git clone https://github.com/UtaUtaUtau/nnsvs-db-converter
!git clone https://github.com/openvpi/DiffSinger.git --branch vr
!git clone https://github.com/openvpi/MakeDiffSinger #for ds training
!git clone https://github.com/MLo7Ghinsan/ghin_shenanigans #for ds segmenting <3 ig and for setup complete wav
!git clone https://github.com/openvpi/SOME #yass some midi estimation

clear_output()
!pip install torch torchvision torchaudio
clear_output()
!pip install -r /content/DiffSinger/requirements.txt
!pip install -r /content/SOME/requirements.txt
clear_output()
!aria2c https://github.com/openvpi/vocoders/releases/download/nsf-hifigan-44.1k-hop512-128bin-2024.02/nsf_hifigan_44.1k_hop512_128bin_2024.02.zip
!aria2c https://github.com/openvpi/DiffSinger/releases/download/v2.1.0/rmvpe.zip
!aria2c https://github.com/openvpi/SOME/releases/download/v1.0.0-baseline/0119_continuous128_5spk.zip
!wget https://github.com/yxlllc/vocal-remover/releases/download/hnsep_240512/hnsep_240512.zip #using wget cus aria2c takes a lot longer for some reason
!unzip -q /content/hnsep_240512.zip -d /content/DiffSinger/checkpoints
!unzip -q 0119_continuous128_5spk.zip -d /content/DiffSinger/checkpoints/SOME
!unzip -q nsf_hifigan_44.1k_hop512_128bin_2024.02.zip -d /content/DiffSinger/checkpoints
!unzip -q /content/rmvpe.zip -d /content/DiffSinger/checkpoints
!unzip -q /content/rmvpe.zip -d /content/MakeDiffSinger/variance-temp-solution/assets
!rm /content/nsf_hifigan_44.1k_hop512_128bin_2024.02.zip
!rm /content/rmvpe.zip
!rm /content/0119_continuous128_5spk.zip

# pretrain stuff
!aria2c -d /content/pretrain_models -o acoustic_pretrain.ckpt https://github.com/haru0l/diffsinger_models/releases/download/acoustic/model_ckpt_steps_49000.ckpt
!aria2c -d /content/pretrain_models -o variance_pretrain.ckpt https://github.com/haru0l/diffsinger_models/releases/download/variance/model_ckpt_steps_51000.ckpt

clear_output()
!pip install --upgrade tensorboard
clear_output()
!pip install protobuf
clear_output()
!pip install onnxruntime
clear_output()
!pip install pydub
clear_output()
#shit tons of clear output cus i dont wanna see anything <3

print("setup complete!")
print("|")
print("|")
print("|")

chika_dance = '<img src="https://raw.githack.com/MLo7Ghinsan/ghin_shenanigans/main/image_and_gif/chika_dance.gif"/>'
display(HTML(chika_dance))

with open("/content/ghin_shenanigans/audio/setup_complete.wav", "rb") as f:
    setup_complete_sound = f.read()
Audio(data=setup_complete_sound, autoplay=True)

# **Preprocess data for training**

In [ ]:
#@title #Extract Data
#@markdown ___
%cd /content
#@markdown this cell will create a folder name [raw_data] in the root folder of colab (/content) and extract your data into it

data_type = "lab + wav (NNSVS format)" # @param ["lab + wav (NNSVS format)", "csv + wav (DiffSinger format)", "ds (DiffSinger format)"]

#@markdown <font size="-1.5"> The path to your data zip file

data_zip_path = "" #@param {type:"string"}

#@markdown ___

#@markdown nnsvs-db-converter settings (lab + wav ONLY)

#@markdown <font size="-1.5"> _These values can exceed the amount that's in your data to maximize the segment length or to keep the data as is_

#@markdown <font size="-1.5"> This option is necessary for variance's pitch training
estimate_midi_option = "False" # @param ["False", "True | parselmouth", "True | harvest", "True | SOME"]
if estimate_midi_option == "True | parselmouth":
    estimate_midi = True
    midi_pitch_ext = "parselmouth"
elif estimate_midi_option == "True | harvest":
    estimate_midi = True
    midi_pitch_ext = "harvest"
else:
    estimate_midi = False
    midi_pitch_ext = None
#@markdown <font size="-1.5"> Determine how long it will segment your data to based on silence phoneme placement (seconds)
segment_length = 15 #@param {type:"slider", min:5, max:35, step:1}

#@markdown <font size="-1.5"> Determine how many silence phoneme is allowed in the middle of each segment
max_silence_phoneme_amount = 2 #@param {type:"slider", min:0, max:50, step:1}

# leaving -S at 60 so max silence can be 60 seconds that exceeds the segment legnth cap idk why///
# making the segment length cap at 35 secs because any longer than that would make training goes really slow

# my ass dont remember why i made two... i think one is unnecessary extra but mehhh
all_shits = "/content/raw_data"
all_shits_not_wav_n_lab = "/content/raw_data/diffsinger_db"

import os
import csv
import json
import shutil
from pydub import AudioSegment

if os.path.exists("/content/raw_data"):
    shutil.rmtree("/content/raw_data")

if not os.path.exists(all_shits_not_wav_n_lab):
  os.makedirs(all_shits_not_wav_n_lab)

# using 'if not' bc i edited the wrong section which im also too lazy to fix it <3
if not data_type == "lab + wav (NNSVS format)":
    #changed to 7zip to support more compression types
    !7z x "$data_zip_path" -o{all_shits_not_wav_n_lab}
    for root, dirs, files in os.walk(all_shits):
        for filename in files:
            if filename.endswith(".lab"):
                file_path = os.path.join(root, filename)
                with open(file_path, "r") as file:
                    file_data = file.read()
                file_data = file_data.replace("SP", "pau")
                file_data = file_data.replace("br", "AP")
                with open(file_path, "w") as file:
                    file.write(file_data)

else:
    !7z x "$data_zip_path" -o{all_shits_not_wav_n_lab}


# for funny auto dict generator lmao
out = "/content/DiffSinger/dictionaries/custom_dict.txt"

phonemes = set()

def is_excluded(phoneme):
    return phoneme in ["pau", "AP", "SP", "sil"]

if data_type == "lab + wav (NNSVS format)":
    phoneme_folder_path = all_shits
    for root, dirs, files in os.walk(phoneme_folder_path):
        for file in files:
            if file.endswith(".lab"):
                fpath = os.path.join(root, file)
                with open(fpath, "r") as lab_file:
                    for line in lab_file:
                        line = line.strip()
                        if line:
                            phoneme = line.split()[2]
                            if not is_excluded(phoneme):
                                phonemes.add(phoneme)
elif data_type == "csv + wav (DiffSinger format)":
    phoneme_folder_path = all_shits_not_wav_n_lab
    for root, dirs, files in os.walk(phoneme_folder_path):
        for file in files:
            if file.endswith(".csv"):
                fpath = os.path.join(root, file)
                with open(fpath, "r", newline="") as csv_file:
                    csv_reader = csv.DictReader(csv_file)
                    for row in csv_reader:
                        if "ph_seq" in row:
                            ph_seq = row["ph_seq"].strip()
                            for phoneme in ph_seq.split():
                                if not is_excluded(phoneme):
                                    phonemes.add(phoneme)

else:
    phoneme_folder_path = all_shits_not_wav_n_lab
    for root, dirs, files in os.walk(phoneme_folder_path):
        for file in files:
            if file.endswith(".ds"):
                fpath = os.path.join(root, file)
                with open(fpath, "r") as json_file:
                    data = json.load(json_file)
                    for entry in data:
                        if "ph_seq" in entry:
                            ph_seq = entry["ph_seq"].strip()
                            phoneme_list = ph_seq.split()
                            for phoneme in phoneme_list:
                                if not is_excluded(phoneme):
                                    phonemes.add(phoneme)

with open(out, "w") as f:
    for phoneme in sorted(phonemes):
        f.write(phoneme + "	" + phoneme + "\n")

# for vowels and consonants.txt.... well adding luquid type for uta's script
dict_path = out
vowel_types = {"a", "i", "u", "e", "o", "N", "M", "NG"}
liquid_types = {"y", "w", "l", "r"} # r for english labels, it should be fine with jp too
vowel_data = []
consonant_data = []
liquid_data = []

with open(dict_path, "r") as f:
    for line in f:
        phoneme, _ = line.strip().split("\t")
        if phoneme[0] in vowel_types:
            vowel_data.append(phoneme)
        elif phoneme[0] in liquid_types:
            liquid_data.append(phoneme)
        else:
            consonant_data.append(phoneme)

vowel_data.sort()
liquid_data.sort()
consonant_data.sort()
directory = os.path.dirname(dict_path)

# make txt for language json file
vowel_txt_path = os.path.join(directory, "vowels.txt")
with open(vowel_txt_path, "w") as f:
    f.write(" ".join(vowel_data))
liquid_txt_path = os.path.join(directory, "liquids.txt")
with open(liquid_txt_path, "w") as f:
    f.write(" ".join(liquid_data))
consonant_txt_path = os.path.join(directory, "consonants.txt")
with open(consonant_txt_path, "w") as f:
    f.write(" ".join(consonant_data))


# here's a funny json append
with open(vowel_txt_path, "r") as f:
    vowel_data = f.read().split()
with open(liquid_txt_path, "r") as f:
    liquid_data = f.read().split()
with open(consonant_txt_path, "r") as f:
    consonant_data = f.read().split()
liquid_list = {liquid: True for liquid in liquid_data} #temp fix, might need more research about the push in timing'''
phones4json = {"vowels": vowel_data, "liquids": liquid_list}
with open("/content/nnsvs-db-converter/lang.sample.json", "w") as rawr:
    json.dump(phones4json, rawr, indent=4)


if data_type == "lab + wav (NNSVS format)":
    db_converter_script = "/content/nnsvs-db-converter/db_converter.py"
    for raw_folder_name in os.listdir(all_shits_not_wav_n_lab):
        raw_folder_path = os.path.join(all_shits_not_wav_n_lab, raw_folder_name)
        if os.path.isdir(raw_folder_path):
            if estimate_midi:
                !python {db_converter_script} -s {max_silence_phoneme_amount} -S 60 -l {segment_length} -m -c -L "/content/nnsvs-db-converter/lang.sample.json" {raw_folder_path}
            else:
                !python {db_converter_script} -s {max_silence_phoneme_amount} -S 60 -l {segment_length} -L "/content/nnsvs-db-converter/lang.sample.json" {raw_folder_path}
            !rm -rf {raw_folder_path}/*.wav {raw_folder_path}/*.lab
            !mv {raw_folder_path}/diffsinger_db/* {raw_folder_path} 2> /dev/null
            !rm -rf {raw_folder_path}/diffsinger_db
            if estimate_midi_option == "True | SOME":
                !python /content/SOME/batch_infer.py --model "/content/DiffSinger/checkpoints/SOME/0119_continuous256_5spk/model_ckpt_steps_100000_simplified.ckpt" --dataset {raw_folder_path} --overwrite

elif data_type == "ds (DiffSinger format)":
    ds_segment_script = "/content/ghin_shenanigans/scripts/ds_segmentor.py"
    ds2csv_script = "/content/MakeDiffSinger/variance-temp-solution/convert_ds.py"
    for raw_folder_name in os.listdir(all_shits_not_wav_n_lab):
        raw_folder_path = os.path.join(all_shits_not_wav_n_lab, raw_folder_name)
        if os.path.isdir(raw_folder_path):
            ds_exp_path = os.path.join(raw_folder_path, "ds")
            csv_exp_path = os.path.join(raw_folder_path, "transcriptions.csv")
            !python {ds_segment_script} {raw_folder_path} --export_path {ds_exp_path}
            !rm -rf {raw_folder_path}/*.ds #clean it cus why not
            !python {ds2csv_script} ds2csv {ds_exp_path} {csv_exp_path}
else:
    pass

# make it replace the first SP to AP cus it seems like people always forgot about it
for root, _, files in os.walk(all_shits_not_wav_n_lab):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            with open(file_path, "r", newline="") as input_file:
                csv_reader = csv.reader(input_file)
                data = [row for row in csv_reader]
                header = data[0]
                if "ph_seq" in header:
                    ph_seq_index = header.index("ph_seq")
                    if len(data) > 1 and len(data[1]) > ph_seq_index:
                        data[1][ph_seq_index] = data[1][ph_seq_index].replace("SP", "AP", 1)
            with open(file_path, "w", newline="") as output_file:
                csv_writer = csv.writer(output_file)
                csv_writer.writerows(data)

print("extraction complete!")
print("|")
print("|")
print("|")
print("I'm also nice enough to convert your data and also write your dict.txt lmao. You are welcome :)")

In [ ]:
#@title #Edit Config
#@markdown ___

import re
import os
import yaml
import random #for the random test files lmaoz

%cd /content
clear_output()
#@markdown <font size="-1.5"> The model type user is training
model_type = "acoustic" # @param ["acoustic", "variance"]
config_cap = model_type.upper()
diffusion_type = "reflow" # @param ["ddpm", "reflow"]
diff_accelerator = "ddim" # @param ["ddim", "pndm", "dpm-solver", "unipc"]
loss_type = "l2" # @param ["l1", "l2"]

spk_name = [folder_name for folder_name in os.listdir(all_shits_not_wav_n_lab) if os.path.isdir(os.path.join(all_shits_not_wav_n_lab, folder_name))]
# i used spk_name for something else cus i forgor now imma just copy and paste it
spk_names = [folder_name for folder_name in os.listdir(all_shits_not_wav_n_lab) if os.path.isdir(os.path.join(all_shits_not_wav_n_lab, folder_name))]
num_spk = len(spk_name)
raw_dir = []
for folder_name in spk_name:
    folder_path = os.path.join(all_shits_not_wav_n_lab, folder_name)
    raw_dir.append(folder_path)
if num_spk == 1:
    singer_type = "SINGLE-SPEAKER"
    use_spk_id = False
    all_wav_files = []
    for root, dirs, files in os.walk("/content/raw_data/diffsinger_db"):
        for file in files:
            if file.endswith((".wav", ".ds")):
                full_path = os.path.join(root, file)
                all_wav_files.append(full_path)
    random.shuffle(all_wav_files)
    random_ass_wavs = all_wav_files[:3]
    random_ass_test_files = [os.path.splitext(os.path.basename(file))[0] for file in random_ass_wavs]

else:
    singer_type = "MULTI-SPEAKER"
    use_spk_id = True
    folder_to_id = {folder_name: i for i, folder_name in enumerate(spk_name)}
    random_ass_test_files = []
    for folder_path in raw_dir:
        if data_type == "ds (DiffSinger format)":
            audio_files = [f[:-4] for f in os.listdir(folder_path) if f.endswith(".ds")]
        else:
            audio_files = [f[:-4] for f in os.listdir(folder_path + "/wavs") if f.endswith(".wav")]
        folder_name = os.path.basename(folder_path)
        folder_id = folder_to_id.get(folder_name, -1)
        prefixed_audio_files = [f"{folder_id}:{audio_file}" for audio_file in audio_files]
        random_ass_test_files.extend(prefixed_audio_files[:3])
spk_id = []
for i, spk_name in enumerate(spk_name):
    spk_id_format = f"{i}:{spk_name}"
    spk_id.append(spk_id_format)

#@markdown <font size="-1.5"> Shallow Diffusion training
use_shallow_diffusion = "true | gt_val" # @param ["false", "true | aux_val", "true | gt_val"]
if use_shallow_diffusion == "false":
    shallow = False
    gt_shallow = False
elif use_shallow_diffusion == "true | aux_val":
    shallow = True
    gt_shallow = False
else:
    shallow = True
    gt_shallow = True

#@markdown <font size="-1.5"> Half precision, or mixed precision can result in improved performance, achieving speedups on training (from [doc](https://lightning.ai/docs/pytorch/stable/common/trainer.html#precision))
# the reason why i dont add 64 is because colab is already dreadfully slow at 32 so yes im leaving it out
precision = "16-mixed" # @param ["32-true", "bf16-mixed", "16-mixed", "bf16", "16"]

#@markdown <font size="-1.5"> User model save path
save_dir = "" #@param {type:"string"}

binary_save_dir = save_dir + "/binary"

conf_dir = save_dir

#@markdown ....................................................................................................................................................................................................................................................................................................................................................................................................................................

#@markdown <font size="-1.5"> Option to use base model for finetuning

enable_finetuning = False # @param {type:"boolean"}


#@markdown <font size="-1.5"> Path to custom base model, leave blank to use [default](https://github.com/haru0l/diffsinger_models) models
#wtf haru i just looked at your readme"""""

base_model_path = "" # @param {type:"string"}

if enable_finetuning:
    pretrain = True
    if base_model_path:
        pretrain_ckpt = base_model_path
    else:
        pretrain_ckpt = f"/content/pretrain_models/{model_type}_pretrain.ckpt"
    finetune_strict_shapes = False
    finetune_ckpt_path = pretrain_ckpt
else:
    pretrain = False
    finetune_strict_shapes = True #default value
    finetune_ckpt_path = None #default value

#@markdown ....................................................................................................................................................................................................................................................................................................................................................................................................................................

#@markdown <font size="-1.5"> Model embeds check; Tension, Energy, Breathiness, Voicing | for both acoustic and variance

#@markdown <font size="-2.5"> we limited the pair up choice to prevent the quality and usage issue, if user wish to enable option(s) outside of these choices then please keep in mind that most of these embeds do not work well together except for [energy + breathiness]

selected_param = "tension" # @param ["energy + breathiness", "tension", "voicing", "none"]
if selected_param ==  "energy + breathiness":
    tension_training = False
    energy_training = True
    breathiness_training = True
    voicing_training = True
elif selected_param == "tension":
    tension_training = True
    energy_training = False
    breathiness_training = False
    voicing_training = False
elif selected_param == "voicing":
    tension_training = False
    energy_training = False
    breathiness_training = False
    voicing_training = True
else:
    tension_training = False
    energy_training = False
    breathiness_training = False
    voicing_training = False

parameter_extraction_method = "vr" # @param ["vr", "world"]

### forcing data aug to be true by default cus i dont think anyone would disable it and its good to be on by default
#markdown <font size="-1.5"> Option to enable "gender" parameter | for acoustic only
data_aug = True #param {type:"boolean"}

#@markdown ....................................................................................................................................................................................................................................................................................................................................................................................................................................

#@markdown <font size="-1.5"> Model training check | for variance only



#@markdown <font size="-2.5"> due to skill issues, if user wish to train with glide embed, please enable it manually in the config
pitch_training = "False" # @param ["False", "True | Standard", "True | MelodyEncoder"]
if pitch_training == "False":
    pitch_training = False
    use_melody_encoder = False
    use_glide_embed = False
elif pitch_training == "True | Standard":
    pitch_training = True
    use_melody_encoder = False
    use_glide_embed = False
else:
    pitch_training = True
    use_melody_encoder = True
    use_glide_embed = False

duration_training = True #@param {type: "boolean"}

#@markdown ....................................................................................................................................................................................................................................................................................................................................................................................................................................

#@markdown <font size="-1.5"> Pitch extractor algorithm

f0_ext = "parselmouth" # @param ["parselmouth", "rmvpe", "harvest"]
if f0_ext == "rmvpe":
    pe_ckpt_pth = "checkpoints/rmvpe/model.pt"
else:
    pe_ckpt_pth = None

#@markdown ....................................................................................................................................................................................................................................................................................................................................................................................................................................

#@markdown <font size="-1.5"> Model's network/layer size, blah blah blah(WIP; does not work just yet, mostly just options and notes for MLo7 for now)

#@markdown <font size="-2.5"> Seems like the quality of samplig_algorithm is in order, range from euler being the LEAST accurate to rk5 being the MOST accurate.... though their tradeoff is the computational power I guess more VRAM usage wooooo!!! i hate it!!!!
sampling_algorithm = "euler" # @param ["euler", "rk2", "rk4", "rk5"]

#@markdown <font size="-2.5"> Hidden layers for FS2 and token param embeds stuff.... doesn't seems to affect the model quality that much though, also changes the model size based on the lower/higher the value
hidden_size = 256 # @param {type:"slider", min:2, max:1024, step:2}

#@markdown <font size="-2.5"> The model's main layers and channels. They determines the final quality so I'd use these with cautions lmaoz
residual_layers = 256 # @param {type:"slider", min:2, max:1024, step:2}
residual_channels = 512 # @param {type:"slider", min:2, max:1024, step:2}

#@markdown <font size="-1.5"> Ill come back and add the rest later ig

with open("/content/DiffSinger/configs/base.yaml", "r") as config:
    mother = yaml.safe_load(config)
mother["pl_trainer_precision"] = precision
with open("/content/DiffSinger/configs/base.yaml", "w") as config:
    yaml.dump(mother, config)

if  data_type == "ds (DiffSinger format)":
    prefer_ds = True
else:
    prefer_ds = False

if model_type == "acoustic":
    with open("/content/DiffSinger/configs/acoustic.yaml", "r") as config:
        bitch_ass_config = yaml.safe_load(config)
    bitch_ass_config["speakers"] = spk_names
    bitch_ass_config["test_prefixes"] = random_ass_test_files
    bitch_ass_config["raw_data_dir"] = raw_dir
    bitch_ass_config["num_spk"] = num_spk
    bitch_ass_config["use_spk_id"] = use_spk_id
    #bitch_ass_config["spk_ids"] = spk_id
    bitch_ass_config["diffusion_type"] = diffusion_type
    bitch_ass_config["diff_accelerator"] = diff_accelerator
    bitch_ass_config["main_loss_type"] = loss_type
    bitch_ass_config["binary_data_dir"] = binary_save_dir
    bitch_ass_config["dictionary"] = "dictionaries/custom_dict.txt"
    bitch_ass_config["augmentation_args"]["random_pitch_shifting"]["enabled"] = data_aug
    bitch_ass_config["augmentation_args"]["random_time_stretching"]["enabled"] = data_aug
    bitch_ass_config["use_key_shift_embed"] = data_aug
    bitch_ass_config["use_speed_embed"] = data_aug
    bitch_ass_config["pe"] = f0_ext
    bitch_ass_config["use_energy_embed"] = energy_training
    bitch_ass_config["use_breathiness_embed"] = breathiness_training
    bitch_ass_config["use_tension_embed"] = tension_training
    bitch_ass_config["use_voicing_embed"] = voicing_training

    bitch_ass_config["pe_ckpt"] = pe_ckpt_pth
    #shallow diff stuff
    bitch_ass_config["use_shallow_diffusion"] = shallow
    bitch_ass_config["shallow_diffusion_args"]["val_gt_start"] = gt_shallow
    #finetue stuff
    bitch_ass_config["finetune_enabled"] = enable_finetuning
    bitch_ass_config["finetune_ckpt_path"] = finetune_ckpt_path
    bitch_ass_config["finetune_strict_shapes"] = finetune_strict_shapes
    #vr
    bitch_ass_config["hnsep"] = parameter_extraction_method
    bitch_ass_config["hnsep_ckpt"] = "checkpoints/vr/model.pt"

    with open("/content/DiffSinger/configs/acoustic.yaml", "w") as config:
        yaml.dump(bitch_ass_config, config)
else:
    with open("/content/DiffSinger/configs/variance.yaml", "r") as config:
        bitch_ass_config = yaml.safe_load(config)
    bitch_ass_config["speakers"] = spk_names
    bitch_ass_config["test_prefixes"] = random_ass_test_files
    bitch_ass_config["raw_data_dir"] = raw_dir
    bitch_ass_config["num_spk"] = num_spk
    bitch_ass_config["use_spk_id"] = use_spk_id
    bitch_ass_config["main_loss_type"] = loss_type
    bitch_ass_config["diffusion_type"] = diffusion_type
    bitch_ass_config["diff_accelerator"] = diff_accelerator
    bitch_ass_config["binary_data_dir"] = binary_save_dir
    bitch_ass_config["dictionary"] = "dictionaries/custom_dict.txt"
    bitch_ass_config["pe"] = f0_ext # i think variance uses it for pitch ref as ground-truth for pitch training soooo
    bitch_ass_config["pe_ckpt"] = pe_ckpt_pth #same goes to this one

    bitch_ass_config["predict_energy"] = energy_training
    bitch_ass_config["predict_breathiness"] = breathiness_training
    bitch_ass_config["predict_tension"] = tension_training
    bitch_ass_config["predict_pitch"] = pitch_training
    bitch_ass_config["predict_voicing"] = voicing_training

    bitch_ass_config["use_melody_encoder"] = use_melody_encoder
    bitch_ass_config["use_glide_embed"] = use_glide_embed
    bitch_ass_config["predict_dur"] = duration_training
    bitch_ass_config["binarization_args"]["prefer_ds"] = prefer_ds
    #finetune stuff
    bitch_ass_config["finetune_enabled"] = enable_finetuning
    bitch_ass_config["finetune_ckpt_path"] = finetune_ckpt_path
    bitch_ass_config["finetune_strict_shapes"] = finetune_strict_shapes
    #vr
    bitch_ass_config["hnsep"] = parameter_extraction_method
    bitch_ass_config["hnsep_ckpt"] = "checkpoints/vr/model.pt"

    with open("/content/DiffSinger/configs/variance.yaml", "w") as config:
        yaml.dump(bitch_ass_config, config)

os.makedirs(save_dir, exist_ok=True)


with open("/content/DiffSinger/utils/hparams.py", "r") as f:
    hparams_py_read = f.read()
hparams_py_read = re.sub(r"args_work_dir\s*=\s*.*", f"args_work_dir = '{save_dir}'", hparams_py_read)
with open("/content/DiffSinger/utils/hparams.py", "w") as f:
    f.write(hparams_py_read)

with open("/content/DiffSinger/utils/training_utils.py", "r") as f:
    training_utils_stuff = f.read()
training_utils_stuff = re.sub("relative_path\s*=\s*.*", "relative_path = filepath.relative_to(Path('/content').resolve())", training_utils_stuff)
with open("/content/DiffSinger/utils/training_utils.py", "w") as f:
    f.write(training_utils_stuff)

print("config updated! see below for config's information")
print("|")
print("|")
print("|")
print(f"+++---{config_cap} {singer_type} TRAINING---+++")
print("|")
print("|")
print("|")
print("+++---user's settings---+++")
print("\n")
print(f"speaker name: {spk_names}")
print("\n")
print(f"data augmentation: {data_aug}")
print("\n")
print(f"pitch extractor: {f0_ext}")
print("\n")
print(f"binary data save directory: {binary_save_dir}")
print("\n")
print(f"your model will be saved to: {save_dir}")
print("\n")
print("==========================================================================================")
print("\n")
print("+++---other auto-defined settings---+++")
print("\n")
print(f"test files (auto selected): {random_ass_test_files}")
print("\n")
print("dictionary (auto generated): custom_dict.txt")
print("\n")
print("==========================================================================================")
print("\n")
print("if you don't like or disagree with any of these options,")
print(f"you can go and edit the config at [/content/DiffSinger/configs/{model_type}.yaml]")


In [ ]:
#@markdown # Preprocess data
import os

# idk i just feel like 800 is a lil low for some people part 2
new_f0_max = 1600
og_script = "/content/DiffSinger/utils/binarizer_utils.py"
with open(og_script, 'r') as file:
    mate = file.read()
up_f0_val = mate.replace("f0_max = 800", f"f0_max = {new_f0_max}")
with open(og_script, 'w') as file:
    file.write(up_f0_val)

training_config = f"/content/DiffSinger/configs/{config_type}.yaml"
%cd /content/DiffSinger
os.environ['PYTHONPATH']='.'
!CUDA_VISIBLE_DEVICES=0 python /content/DiffSinger/scripts/binarize.py --config {training_config} --reset

# **Training**

In [ ]:
#@markdown #Train your model
%cd /content/DiffSinger
import re
import os
import yaml
#@markdown ___

#@markdown <font size="-1.5"> Step interval of when your model will be validate and save
save_interval = 2000 #@param {type:"slider", min:100, max:10000, step:100}

#@markdown <font size="-1.5"> batch size setting, too low can cause bottleneck, too high can cause oom
batch_size = 9 # @param {type:"slider", min:1, max:100, step:1}

#@markdown ___

#@markdown ###**Only edit this section if you want to resume training**
resume_training = False #@param {type:"boolean"}

#@markdown <font size="-1.5"> select this option if you locally binarized your data | this option will only append your binary data path in your config | "binary" folder must be in the same directory as config.yaml
local_data = False #@param {type:"boolean"}

#@markdown <font size="-1.5"> path to the config you got from training
re_config_path = "" #@param {type:"string"}
model_dir = os.path.dirname(re_config_path)
save_dir = model_dir
if resume_training:
    with open("/content/DiffSinger/utils/hparams.py", "r") as f:
        hparams_py_read = f.read()
    hparams_py_read = re.sub(r"args_work_dir\s*=\s*.*", f"args_work_dir = '{save_dir}'", hparams_py_read)
    with open("/content/DiffSinger/utils/hparams.py", "w") as f:
        f.write(hparams_py_read)
    with open("/content/DiffSinger/utils/training_utils.py", "r") as f:
        training_utils_stuff = f.read()
    training_utils_stuff = re.sub("relative_path\s*=\s*.*", "relative_path = filepath.relative_to(Path('/content').resolve())", training_utils_stuff)
    with open("/content/DiffSinger/utils/training_utils.py", "w") as f:
        f.write(training_utils_stuff)

    config_path = re_config_path
    log_dir = save_dir

    !cp {model_dir}/dictionary.txt /content/DiffSinger/dictionaries/custom_dict.txt

else:
    config_path = training_config
    log_dir = conf_dir

with open(config_path, "r") as config:
    ehe = yaml.safe_load(config)
config_dir = os.path.dirname(config_path)
yuh = os.path.join(config_dir, "binary")

ehe["val_check_interval"] = save_interval
ehe["max_batch_size"] = batch_size
if local_data:
    ehe["binary_data_dir"] = yuh
with open(config_path, "w") as config:
    yaml.dump(ehe, config)

logs = log_dir
%reload_ext tensorboard
%tensorboard --logdir {logs}/lightning_logs

!python /content/DiffSinger/scripts/train.py --config {config_path} --exp_name ${save_dir} --reset

# **Convert model to ONNX format**

In [ ]:
#@markdown # Drop Speakers from Model (Optional)
#@markdown ___
#@markdown <font size="-1.5"> Use this to drop speakers from your model for distribution. You will need to do it for both acoustic and variance models.

drop_model_path = '' #@param {type: "string"}
#@markdown <font size="-1.5"> Type the ID of speakers you'd like to KEEP separated by commas. Ex: "0,3,4" <br>
#@markdown <font size="-1.5"> Note: You can find the ID of speakers in the model by opening the ```spk_map.json``` file in the model folder.<br>
#@markdown <font size="-1.5"> If you see ```{"natural": 0, "power": 1, "silly": 2}``` but only want to keep "natural" and "power", type ```0,1``` below.
retain_speakers = '' #@param {type: "string"}
#@markdown <font size="-1.5"> If you don't know what this means, don't change it.
fill_embed = 'zeros' #@param ['zeros', 'random', 'mean', 'cyclic']

drop_out_path = drop_model_path[:-5] + '_spk-dropped.ckpt'

!python /content/DiffSinger/scripts/drop_spk.py {drop_model_path} {drop_out_path} --retain {retain_speakers} --fill {fill_embed}




In [ ]:
#@markdown # Export ONNX
#@markdown ___
%cd /content
from IPython.display import clear_output
clear_output()
import os
import zipfile
import shutil

#move this here so users doesnt have to wait for this to install for training
print("installing dependencies for onnx conversion...")
!pip install torch==1.13.0 torchvision==0.14.0 torchaudio==0.13.0 -q -q -q 2>/dev/null

# to counter IF the user is to re-run this cell <3
if os.path.exists("/content/OU_compatible_files"):
    shutil.rmtree("/content/OU_compatible_files")
    os.remove("/content/jpn_dict.txt")
else:
    pass

#@markdown <font size="-1.5"> select this if you don't want to see the onnx converter's output
no_output = True # @param {type:"boolean"}

#@markdown <font size="-1.5"> path to your **ACOUSTIC CHECKPOINT** (leave blank if you don't have any): automatically use latest checkpoint that is in the same folder
acoustic_checkpoint_path = "" #@param{type:"string'}
acoustic_folder_name = os.path.basename(os.path.dirname(acoustic_checkpoint_path)) + "_acoustic"
acoustic_folder_path = os.path.dirname(acoustic_checkpoint_path)

#@markdown <font size="-1.5"> path to your **VARIANCE CHECKPOINT** (leave blank if you don't have any): automatically use latest checkpoint that is in the same folder
variance_checkpoint_path = "" #@param{type:"string'}
variance_folder_name = os.path.basename(os.path.dirname(variance_checkpoint_path)) + "_variance"
variance_folder_path = os.path.dirname(variance_checkpoint_path)

#@markdown <font size="-1.5"> path to where you want to save your ONNX files (it will create a folder named "onnx" in this path)
exp_folder = "" #@param{type:"string"}

acoustic_onnx_exp = exp_folder + "/onnx/acoustic"
variance_onnx_exp = exp_folder + "/onnx/variance"

if not acoustic_checkpoint_path:
    print("\n")
    print("acoustic ckeckpoint path not specified, not exporting acoustic ONNX...")
else:
    print("\n")
    print("converting acoustic to onnx...")
    #cp stuff cus apparently exporter doesnt work without it
    !cp {acoustic_folder_path}/config.yaml -r /content/DiffSinger/checkpoints/{acoustic_folder_name}
    search_text = "        args_work_dir = os.path.join("
    replacement = f"        args_work_dir = '{acoustic_folder_path}'"
    with open("/content/DiffSinger/utils/hparams.py", "r") as file:
        lines = file.readlines()
    for i, line in enumerate(lines):
        if search_text in line:
            lines[i] = replacement + "\n"
            break
    with open("/content/DiffSinger/utils/hparams.py", "w") as file:
            file.writelines(lines)
    #incase if anyone wanna change it lmao
    search_text_alt = "        args_work_dir = '"
    replacement_alt = f"        args_work_dir = '{acoustic_folder_path}'"
    with open("/content/DiffSinger/utils/hparams.py", "r") as file:
        lines = file.readlines()
    for i, line in enumerate(lines):
        if search_text_alt in line:
            lines[i] = replacement_alt + "\n"
            break
    with open("/content/DiffSinger/utils/hparams.py", "w") as file:
            file.writelines(lines)

    if no_output:
        !python /content/DiffSinger/scripts/export.py acoustic --exp {acoustic_folder_name} --out {exp_folder}/onnx/acoustic >/dev/null 2>&1
    else:
        !python /content/DiffSinger/scripts/export.py acoustic --exp {acoustic_folder_name} --out {exp_folder}/onnx/acoustic


if not variance_checkpoint_path:
    print("\n")
    print("variance ckeckpoint path not specified, not exporting variance ONNX...")
else:
    print("\n")
    print("converting variance to onnx...")
    #cp stuff cus apparently exporter doesnt work without it
    !cp {variance_folder_path}/config.yaml -r /content/DiffSinger/checkpoints/{variance_folder_name}
    search_text = "        args_work_dir = os.path.join("
    replacement = f"        args_work_dir = '{variance_folder_path}'"
    with open("/content/DiffSinger/utils/hparams.py", "r") as file:
        lines = file.readlines()
    for i, line in enumerate(lines):
        if search_text in line:
            lines[i] = replacement + "\n"
            break
    with open("/content/DiffSinger/utils/hparams.py", "w") as file:
            file.writelines(lines)
    #incase if anyone wanna change it lmao
    search_text_alt = "        args_work_dir = '"
    replacement_alt = f"        args_work_dir = '{variance_folder_path}'"
    with open("/content/DiffSinger/utils/hparams.py", "r") as file:
        lines = file.readlines()
    for i, line in enumerate(lines):
        if search_text_alt in line:
            lines[i] = replacement_alt + "\n"
            break
    with open("/content/DiffSinger/utils/hparams.py", "w") as file:
            file.writelines(lines)
    if no_output:
        !python /content/DiffSinger/scripts/export.py variance --exp {variance_folder_name} --out {exp_folder}/onnx/variance >/dev/null 2>&1
    else:
        !python /content/DiffSinger/scripts/export.py variance --exp {variance_folder_name} --out {exp_folder}/onnx/variance


if not variance_checkpoint_path:
    folder_paths = [acoustic_onnx_exp]
elif not acoustic_checkpoint_path:
    folder_paths = [variance_onnx_exp]
else:
    folder_paths = [acoustic_onnx_exp, variance_onnx_exp]

patterns = {"acoustic.onnx": "acoustic.onnx", "dur.onnx": "dur.onnx", "linguistic.onnx": "linguistic.onnx", "pitch.onnx": "pitch.onnx", "variance.onnx": "variance.onnx", "phonemes.txt": "phonemes.txt"}

for folder_path in folder_paths:
    for filename in os.listdir(folder_path):
        for pattern, new_name in patterns.items():
            if pattern in filename:
                old_path = os.path.join(folder_path, filename)
                new_path = os.path.join(folder_path, new_name)
                if os.path.exists(old_path):
                    os.rename(old_path, new_path)
for folder_path in folder_paths:
    for filename in os.listdir(folder_path):
        if "acoustic_acoustic." in filename:
            new_filename = filename.replace("acoustic_acoustic.", "acoustic_")
        elif "variance_variance." in filename:
            new_filename = filename.replace("variance_variance.", "variance_")
        else:
            new_filename = filename
        old_path = os.path.join(folder_path, filename)
        new_path = os.path.join(folder_path, new_filename)
        os.rename(old_path, new_path)
print("\n")
print("ONNX export complete! Please refer to https://github.com/xunmengshe/OpenUtau/wiki/Voicebank-Development to make your model OU compatible")
print("\n")
print("Or use the 'Build OpenUtau VB' cell to have things set up for you")


# **Miscellaneous**

In [ ]:
#@title #Raw data conversion
#@markdown ___
%cd /content
#@markdown This cell will export .lab and .ds files along with your data

data_type = "lab + wav (NNSVS format)" # @param ["lab + wav (NNSVS format)"]

#@markdown <font size="-1.5"> The path to your data zip file

data_zip_path = "" #@param {type:"string"}

#@markdown <font size="-1.5"> The path you will be saving the data to

data_save_path = "" #@param {type:"string"}

#@markdown ___

export_ds = True

#@markdown <font size="-1.5"> _These values can exceed the amount that's in your data to maximize the segment length or to keep the data as is_

#@markdown <font size="-1.5"> Determine how long it will segment your data to based on silence phoneme placement (seconds)
segment_length = 15 #@param {type:"slider", min:5, max:35, step:1}

#@markdown <font size="-1.5"> Determine how many silence phoneme is allowed in the middle of each segment
max_silence_phoneme_amount = 2 #@param {type:"slider", min:0, max:50, step:1}

# leaving -S at 60 so max silence can be 60 seconds that exceeds the segment legnth cap idk why///
# making the segment length cap at 35 secs because any longer than that would make training goes really slow

# my ass dont remember why i made two... i think one is unnecessary extra but mehhh
all_shits = "/content/raw_data"
all_shits_not_wav_n_lab = "/content/raw_data/diffsinger_db"

import os
import csv
import json
import shutil
from pydub import AudioSegment

if os.path.exists("/content/raw_data"):
    shutil.rmtree("/content/raw_data")

if not os.path.exists(all_shits_not_wav_n_lab):
  os.makedirs(all_shits_not_wav_n_lab)

# using 'if not' bc i edited the wrong section which im also too lazy to fix it <3
if not data_type == "lab + wav (NNSVS format)":
    #changed to 7zip to support more compression types
    !7z x "$data_zip_path" -o{all_shits_not_wav_n_lab}
    for root, dirs, files in os.walk(all_shits):
        for filename in files:
            if filename.endswith(".lab"):
                file_path = os.path.join(root, filename)
                with open(file_path, "r") as file:
                    file_data = file.read()
                file_data = file_data.replace("SP", "pau")
                file_data = file_data.replace("br", "AP")
                with open(file_path, "w") as file:
                    file.write(file_data)

else:
    !7z x "$data_zip_path" -o{all_shits_not_wav_n_lab}


# for funny auto dict generator lmao
out = "/content/raw_data/custom_dict.txt"

phonemes = set()

def is_excluded(phoneme):
    return phoneme in ["pau", "AP", "SP"]

if data_type == "lab + wav (NNSVS format)":
    phoneme_folder_path = all_shits
    for root, dirs, files in os.walk(phoneme_folder_path):
        for file in files:
            if file.endswith(".lab"):
                fpath = os.path.join(root, file)
                with open(fpath, "r") as lab_file:
                    for line in lab_file:
                        line = line.strip()
                        if line:
                            phoneme = line.split()[2]
                            if not is_excluded(phoneme):
                                phonemes.add(phoneme)

with open(out, "w") as f:
    for phoneme in sorted(phonemes):
        f.write(phoneme + "	" + phoneme + "\n")

# for vowels and consonants.txt.... well adding liquid type for uta's script
dict_path = out
vowel_types = {"a", "i", "u", "e", "o", "N", "M", "NG"}
liquid_types = {"y", "w", "l", "r"} # r for english labels, it should be fine with jp too
vowel_data = []
consonant_data = []
liquid_data = []

with open(dict_path, "r") as f:
    for line in f:
        phoneme, _ = line.strip().split("\t")
        if phoneme[0] in vowel_types:
            vowel_data.append(phoneme)
        elif phoneme[0] in liquid_types:
            liquid_data.append(phoneme)
        else:
            consonant_data.append(phoneme)

vowel_data.sort()
liquid_data.sort()
consonant_data.sort()
directory = os.path.dirname(dict_path)

# make txt for language json file
vowel_txt_path = os.path.join(directory, "vowels.txt")
with open(vowel_txt_path, "w") as f:
    f.write(" ".join(vowel_data))
liquid_txt_path = os.path.join(directory, "liquids.txt")
with open(liquid_txt_path, "w") as f:
    f.write(" ".join(liquid_data))
consonant_txt_path = os.path.join(directory, "consonants.txt")
with open(consonant_txt_path, "w") as f:
    f.write(" ".join(consonant_data))


# here's a funny json append
with open(vowel_txt_path, "r") as f:
    vowel_data = f.read().split()
with open(liquid_txt_path, "r") as f:
    liquid_data = f.read().split()
with open(consonant_txt_path, "r") as f:
    consonant_data = f.read().split()
phones4json = {"vowels": vowel_data, "liquids": liquid_data}
with open("/content/nnsvs-db-converter/lang.sample.json", "w") as rawr:
    json.dump(phones4json, rawr, indent=4)


if data_type == "lab + wav (NNSVS format)":
    db_converter_script = "/content/nnsvs-db-converter/db_converter.py"
    for raw_folder_name in os.listdir(all_shits_not_wav_n_lab):
        raw_folder_path = os.path.join(all_shits_not_wav_n_lab, raw_folder_name)
        if os.path.isdir(raw_folder_path):
            !python {db_converter_script} -s {max_silence_phoneme_amount} -S 60 -l {segment_length} ${export_lab} -mD -c -L "/content/nnsvs-db-converter/lang.sample.json" -w htk --folder {raw_folder_path}

if data_type == "lab + wav (NNSVS format)":
    for raw_folder_name in os.listdir(all_shits_not_wav_n_lab):
        raw_folder_path = os.path.join(all_shits_not_wav_n_lab, raw_folder_name)
        !rm -rf {raw_folder_path}/*.wav {raw_folder_path}/*.lab
        !mv {raw_folder_path}/diffsinger_db/* {raw_folder_path} 2> /dev/null
        !rm -rf {raw_folder_path}/diffsinger_db
        #!cp {raw_folder_path}/wavs/*.wav {raw_folder_path}

# make it replace the first SP to AP cus it seems like people always forgot about it
for root, _, files in os.walk(all_shits_not_wav_n_lab):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            with open(file_path, "r", newline="") as input_file:
                csv_reader = csv.reader(input_file)
                data = [row for row in csv_reader]
                header = data[0]
                if "ph_seq" in header:
                    ph_seq_index = header.index("ph_seq")
                    if len(data) > 1 and len(data[1]) > ph_seq_index:
                        data[1][ph_seq_index] = data[1][ph_seq_index].replace("SP", "AP", 1)
            with open(file_path, "w", newline="") as output_file:
                csv_writer = csv.writer(output_file)
                csv_writer.writerows(data)

print("extraction complete!")
print("\n")
print("zipping up files...")
!zip -q -9 -r {data_save_path}/data.zip /content/raw_data/*

In [ ]:
#@markdown # Build OpenUtau VB
#@markdown ___
#i need to clean this up it seems
#plan: add a build ou section here by inserting onnx paths (or just the folder containing the folders to the onnx files) to build ou
# ill have a config read function too so i dont have to add checkmark of if people train with embeds or shallow diff or not <3
# yes im lazy rawr x3
%cd /content
import os
import shutil
import yaml

constr_folder = "/content/OU_voicebank"
if not os.path.exists(constr_folder):
    os.makedirs(constr_folder)
else:
    shutil.rmtree(constr_folder)
    os.makedirs(constr_folder)

clear_output()

#@markdown <font size="-1.5"> path to your **ACOUSTIC ONNX FOLDER**
acoustic_onnx_folder = "" #@param{type:"string'}
#@markdown <font size="-1.5"> path to the config.yaml of acoustic model
acoustic_config = "" #@param{type:"string'}

#@markdown <font size="-1.5"> path to your **VARIANCE ONNX FOLDER**
variance_onnx_folder = "" #@param{type:"string'}
#@markdown <font size="-1.5"> path to the config.yaml of variance model
variance_config = "" #@param{type:"string'}

#@markdown <font size="-1.5"> path to your word to phoneme dict (leave blank to use default Japanese dict)
dictionary_path = "" #@param{type:"string"}

#@markdown <font size="-1.5"> path to the folder you want to save the zip file to
save_path = "" #@param{type:"string"}

#@markdown ___

#@markdown ## Character Configuration | character.txt and character.yaml

#@markdown <font size="-1.5"> your character display name| **required**
name = "" #@param{type:"string"}

print("copying files...")
main_stuff = f"{constr_folder}/{name}"
if not os.path.exists(main_stuff):
    os.makedirs(main_stuff)
if not os.path.exists(f"{main_stuff}/dsmain"):
    os.makedirs(f"{main_stuff}/dsmain/embeds/acoustic")
    os.makedirs(f"{main_stuff}/dsmain/embeds/variance")
!cp {acoustic_onnx_folder}/acoustic.onnx {main_stuff}/dsmain
!cp {acoustic_onnx_folder}/phonemes.txt {main_stuff}/dsmain
!cp {acoustic_onnx_folder}/*.emb {main_stuff}/dsmain/embeds/acoustic >/dev/null 2>&1
!cp {variance_onnx_folder}/*.emb {main_stuff}/dsmain/embeds/variance >/dev/null 2>&1

if variance_onnx_folder:
    !cp {variance_onnx_folder}/linguistic.onnx {main_stuff}/dsmain
else:
    pass

print("\n")
print("writing character.txt...")
with open(f"{main_stuff}/character.txt", "w") as file:
    file.write(f"name={name}\n")
    file.write("image=\n")
    file.write("author=\n")
    file.write("voice=\n")
    file.write("web=\n")

print("\n")
print("writing character.yaml...")
with open(f"{main_stuff}/character.yaml", "w") as file:
    file.write("text_file_encoding: utf-8\n")
    file.write("portrait:\n")
    file.write("portrait_opacity: 0.45\n")
    file.write("default_phonemizer: OpenUtau.Core.DiffSinger.DiffSingerPhonemizer\n")
    file.write("singer_type: diffsinger\n")
acoustic_emb_files = os.listdir(acoustic_onnx_folder)
acoustic_embeds = []
acoustic_color_suffix = []
for file in acoustic_emb_files:
    if file.endswith(".emb"):
        acoustic_emb = os.path.splitext(file)[0]
        acoustic_embeds.append("dsmain/embeds/acoustic/" + acoustic_emb)
        acoustic_color_suffix.append(acoustic_emb)
subbanks = []
for i, (acoustic_embed_color, acoustic_embed_suffix) in enumerate(zip(acoustic_color_suffix, acoustic_embeds), start=1):
    color = f"{i:02}: {acoustic_embed_color}"
    suffix = f"{acoustic_embed_suffix}"
    subbanks.append({"color": color, "suffix": suffix})
if subbanks:
    with open(f"{main_stuff}/character.yaml", "r") as config:
        i_wanna_die_slash_j = yaml.safe_load(config)
    i_wanna_die_slash_j["subbanks"] = subbanks
    with open(f"{main_stuff}/character.yaml", "w") as config:
        yaml.dump(i_wanna_die_slash_j, config)

print("\n")
print("writing dsconfig.yaml for acoustic...")
with open(f"{main_stuff}/dsconfig.yaml", "w") as file:
    file.write("phonemes: dsmain/phonemes.txt\n")
    file.write("acoustic: dsmain/acoustic.onnx\n")
    file.write("vocoder: nsf_hifigan\n")
    file.write("singer_type: diffsinger\n")
with open(acoustic_config, "r") as config:
    mfking_config = yaml.safe_load(config)
use_energy_embed = mfking_config.get("use_energy_embed")
use_breathiness_embed = mfking_config.get("use_breathiness_embed")
use_shallow_diffusion = mfking_config.get("use_shallow_diffusion")
max_depth = mfking_config.get("T_start")
speakers = mfking_config.get("speakers") #looking back here, why is this even here lmao cus i used acoustic_embeds instead of speakers
augmentation_arg = mfking_config.get("augmentation_args")
pitch_aug = mfking_config.get("use_key_shift_embed")
time_aug = mfking_config.get("use_speed_embed")
voicing = mfking_config.get("use_voicing_embed")
tension = mfking_config.get("use_tension_embed")
sample_rate = mfking_config.get("audio_sample_rate")
hop_size = mfking_config.get("hop_size")
win_size = mfking_config.get("win_size")
fft_size = mfking_config.get("fft_size")
num_mel_bins = mfking_config.get("audio_num_mel_bins")
mel_fmin = mfking_config.get("fmin")
mel_fmax = mfking_config.get("fmax")
mel_base = mfking_config.get("mel_base")

with open(f"{main_stuff}/dsconfig.yaml", "r") as config:
    why_are_there_so_many_i_could_prob_make_it_one = yaml.safe_load(config)
why_are_there_so_many_i_could_prob_make_it_one["use_energy_embed"] = use_energy_embed
why_are_there_so_many_i_could_prob_make_it_one["use_breathiness_embed"] = use_breathiness_embed
why_are_there_so_many_i_could_prob_make_it_one["use_variable_depth"] = use_shallow_diffusion
why_are_there_so_many_i_could_prob_make_it_one["max_depth"] = max_depth
why_are_there_so_many_i_could_prob_make_it_one["augmentation_args"] = augmentation_arg
why_are_there_so_many_i_could_prob_make_it_one["use_key_shift_embed"] = pitch_aug
why_are_there_so_many_i_could_prob_make_it_one["use_speed_embed"] = time_aug
why_are_there_so_many_i_could_prob_make_it_one["use_voicing_embed"] = voicing
why_are_there_so_many_i_could_prob_make_it_one["use_tension_embed"] = tension
why_are_there_so_many_i_could_prob_make_it_one["use_continuous_acceleration"] = True
why_are_there_so_many_i_could_prob_make_it_one["sample_rate"] = sample_rate
why_are_there_so_many_i_could_prob_make_it_one["hop_size"] = hop_size
why_are_there_so_many_i_could_prob_make_it_one["win_size"] = win_size
why_are_there_so_many_i_could_prob_make_it_one["fft_size"] = fft_size
why_are_there_so_many_i_could_prob_make_it_one["num_mel_bins"] = num_mel_bins
why_are_there_so_many_i_could_prob_make_it_one["fmin"] = mel_fmin
why_are_there_so_many_i_could_prob_make_it_one["fmax"] = mel_fmax
why_are_there_so_many_i_could_prob_make_it_one["mel_base"] = mel_base
why_are_there_so_many_i_could_prob_make_it_one["mel_scale"] = "slaney"


if subbanks:
    why_are_there_so_many_i_could_prob_make_it_one["speakers"] = acoustic_embeds
with open(f"{main_stuff}/dsconfig.yaml", "w") as config:
    yaml.dump(why_are_there_so_many_i_could_prob_make_it_one, config)


variance_emb_files = os.listdir(variance_onnx_folder)
variance_embeds = []
for file in variance_emb_files:
    if file.endswith(".emb"):
        variance_emb = os.path.splitext(file)[0]
        variance_embeds.append("../dsmain/embeds/variance/" + variance_emb)

print("\n")
print("writing dsdict.yaml...")
if not dictionary_path:
    dict_path = "/content/jpn_dict.txt"
else:
    dict_path = dictionary_path

# for symbols list
phoneme_dict_path = f"{acoustic_onnx_folder}/dictionary.txt"

dsdict = "dsdict.yaml"

def parse_phonemes(phonemes_str):
    return phonemes_str.split()

entries = []
vowel_types = {"a", "i", "u", "e", "o", "N", "M", "NG", "cl", "vf"}
vowel_data = []
stop_data = []

# Process the specified dictionary
with open(dict_path, "r") as f:
    for line in f:
        word, phonemes_str = line.strip().split("\t")
        phonemes = parse_phonemes(phonemes_str)
        if len(phonemes) == 1:
            entries.append({"grapheme": word, "phonemes": phonemes})
        else:
            entries.append({"grapheme": word, "phonemes": phonemes})

with open(phoneme_dict_path, "r") as f:
    for line in f:
        phoneme, _ = line.strip().split("\t")
        phoneme_type = "vowel" if phoneme[0] in vowel_types else "stop"
        entry = {"symbol": phoneme, "type": phoneme_type}
        if phoneme_type == "vowel":
            vowel_data.append(entry)
        else:
            stop_data.append(entry)

vowel_data.sort(key=lambda x: x["symbol"])
stop_data.sort(key=lambda x: x["symbol"])

dsdict_path = os.path.join(constr_folder, dsdict)
with open(dsdict_path, "w") as f:
    f.write("entries:\n")
    for entry in entries:
        f.write(f"- grapheme: {entry['grapheme']}\n")
        f.write("  phonemes:\n")
        for phoneme in entry["phonemes"]:
            f.write(f"  - {phoneme}\n")

    f.write("\nsymbols:\n")
    for entry in vowel_data + stop_data:
        f.write(f"- symbol: {entry['symbol']}\n")
        f.write(f"  type: {entry['type']}\n")

with open(variance_config, "r") as config:
    mfking_config = yaml.safe_load(config)
sample_rate = mfking_config.get("audio_sample_rate")
hop_size = mfking_config.get("hop_size")
predict_dur = mfking_config.get("predict_dur")
predict_pitch = mfking_config.get("predict_pitch")
use_melody_encoder = mfking_config.get("use_melody_encoder")
predict_voicing = mfking_config.get("predict_voicing")
predict_tension = mfking_config.get("predict_tension")
predict_energy = mfking_config.get("predict_energy")
predict_breathiness = mfking_config.get("predict_breathiness")

dur_onnx_path = variance_onnx_folder + "/dur.onnx"
if os.path.exists(dur_onnx_path):
    print("\n")
    print("making dsdur directory and necessary files...")
    os.makedirs(f"{main_stuff}/dsdur")
    !cp {dur_onnx_path} {main_stuff}/dsdur
    !cp {dsdict_path} {main_stuff}/dsdur
    with open(f"{main_stuff}/dsdur/dsconfig.yaml", "w") as file:
        file.write("phonemes: ../dsmain/phonemes.txt\n")
        file.write("linguistic: ../dsmain/linguistic.onnx\n")
        file.write("dur: dur.onnx\n")
    with open(f"{main_stuff}/dsdur/dsconfig.yaml", "r") as config:
        dsdur_config = yaml.safe_load(config)
    dsdur_config["use_continuous_acceleration"] = True
    dsdur_config["sample_rate"] = sample_rate
    dsdur_config["hop_size"] = hop_size
    dsdur_config["predict_dur"] = predict_dur
    if subbanks:
        dsdur_config["speakers"] = variance_embeds
    with open(f"{main_stuff}/dsdur/dsconfig.yaml", "w") as config:
        yaml.dump(dsdur_config, config)
else:
    print("\n")
    print("dur.onnx not found, skipping on making dsdur folder...")

pitch_onnx_path = variance_onnx_folder + "/pitch.onnx"
if os.path.exists(pitch_onnx_path):
    print("\n")
    print("making dspitch directory and necessary files...")
    os.makedirs(f"{main_stuff}/dspitch")
    !cp {pitch_onnx_path} {main_stuff}/dspitch
    !cp {dsdict_path} {main_stuff}/dspitch
    with open(f"{main_stuff}/dspitch/dsconfig.yaml", "w") as file:
        file.write("phonemes: ../dsmain/phonemes.txt\n")
        file.write("linguistic: ../dsmain/linguistic.onnx\n")
        file.write("pitch: pitch.onnx\n")
        file.write("use_expr: true\n")
    with open(f"{main_stuff}/dspitch/dsconfig.yaml", "r") as config:
        dspitch_config = yaml.safe_load(config)
    dspitch_config["use_continuous_acceleration"] = True
    dspitch_config["sample_rate"] = sample_rate
    dspitch_config["hop_size"] = hop_size
    dspitch_config["predict_dur"] = predict_pitch
    if subbanks:
        dspitch_config["speakers"] = variance_embeds
    dspitch_config["use_note_rest"] = use_melody_encoder
    with open(f"{main_stuff}/dspitch/dsconfig.yaml", "w") as config:
        yaml.dump(dspitch_config, config)
else:
    print("\n")
    print("pitch.onnx not found, skipping on making dspitch folder...")

variance_onnx_path = variance_onnx_folder + "/variance.onnx"
if os.path.exists(variance_onnx_path):
    print("\n")
    print("making dsvariance directory and necessary files...")
    os.makedirs(f"{main_stuff}/dsvariance")
    !cp {variance_onnx_path} {main_stuff}/dsvariance
    !cp {dsdict_path} {main_stuff}/dsvariance
    with open(f"{main_stuff}/dsvariance/dsconfig.yaml", "w") as file:
        file.write("phonemes: ../dsmain/phonemes.txt\n")
        file.write("linguistic: ../dsmain/linguistic.onnx\n")
        file.write("variance: variance.onnx\n")
    with open(f"{main_stuff}/dsvariance/dsconfig.yaml", "r") as config:
        dsvariance_config = yaml.safe_load(config)
    dsvariance_config["use_continuous_acceleration"] = True
    dsvariance_config["sample_rate"] = sample_rate
    dsvariance_config["hop_size"] = hop_size
    dsvariance_config["predict_dur"] = True #this one will always be true cus if there's no variance model, it shouldnt make this folder in the first place
    dsvariance_config["predict_voicing"] = predict_voicing
    dsvariance_config["predict_tension"] = predict_tension
    dsvariance_config["predict_energy"] = predict_energy
    dsvariance_config["predict_breathiness"] = predict_breathiness
    if subbanks:
        dsvariance_config["speakers"] = variance_embeds
    with open(f"{main_stuff}/dsvariance/dsconfig.yaml", "w") as config:
        yaml.dump(dsvariance_config, config)
else:
    print("\n")
    print("variance.onnx not found, skipping on making dsvariance folder...")

!rm -rf {dsdict_path}
#im too lazy to write codes so ill just do this, itll only remove those folders if they're empty anyway
!rm -d {main_stuff}/dsmain/embeds/* >/dev/null 2>&1
!rm -d {main_stuff}/dsmain/embeds >/dev/null 2>&1

print("\n")
print("zipping up files...")
!zip -q -9 -r {save_path}/{name}.zip {main_stuff}/*

print("\n")
print("done!")

print("\n")
print("You can download your model zip and use it in OpenUtau! If anything needed to be edit in the config then please do so")